# Feature Engineering

## Import the libraries

In [27]:
import pandas as pd
import numpy as np
from textblob import TextBlob
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Create the Dataframe

In [28]:
df_reviews = pd.read_csv(r'Datasets\user_reviews_clean.csv')

In [29]:
df_items = pd.read_csv(r'Datasets\user_items_clean.csv')

In [30]:
df_games = pd.read_csv(r'Datasets\steam_games_clean.csv')

In [31]:
df_reviews

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,maplemage,http://steamcommunity.com/id/maplemage,3 people found this review funny,"Posted April 15, 2014.",NaN,211420,35 of 43 people (81%) found this review helpful,True,Git gud
1,maplemage,http://steamcommunity.com/id/maplemage,1 person found this review funny,"Posted December 23, 2013.",NaN,211820,12 of 16 people (75%) found this review helpful,True,"It's like Terraria, you play for 9 hours strai..."
2,maplemage,http://steamcommunity.com/id/maplemage,2 people found this review funny,"Posted March 14, 2014.",NaN,730,5 of 5 people (100%) found this review helpful,True,"Hold shift to win, Hold CTRL to lose."
3,Wackky,http://steamcommunity.com/id/Wackky,1 person found this review funny,"Posted October 21, 2012.","Last edited November 25, 2013.",550,1 of 1 people (100%) found this review helpful,True,This game is fantastic if you are looking to D...
4,76561198079601835,http://steamcommunity.com/profiles/76561198079...,1 person found this review funny,Posted May 20.,NaN,730,0 of 1 people (0%) found this review helpful,True,ZIKA DO BAILE
...,...,...,...,...,...,...,...,...,...
8005,MeloncraftLP,http://steamcommunity.com/id/MeloncraftLP,2 people found this review funny,Posted July 27.,NaN,265630,1 of 15 people (7%) found this review helpful,False,DON'T INSTALL THIS GAME!!!!!!!!!!!!!!!!!!!!!!!...
8006,76561198277602337,http://steamcommunity.com/profiles/76561198277...,3 people found this review funny,Posted August 2.,NaN,306130,4 of 19 people (21%) found this review helpful,False,"Game is terrible! So, Iv tried 15hrs tryin to ..."
8007,sexyawp,http://steamcommunity.com/id/sexyawp,1 person found this review funny,Posted April 25.,NaN,427730,1 of 2 people (50%) found this review helpful,True,dont ask
8008,76561198310819422,http://steamcommunity.com/profiles/76561198310...,1 person found this review funny,Posted June 23.,NaN,570,1 of 1 people (100%) found this review helpful,True,Well Done


## Sentiment Analysis

We make a function to analyze the sentiments in df_reviews, we will apply it in the reviews column to make a new column and delete the other, to simplify the work of the machine learning

In [32]:
def analyze_sentiment(text):
    if text is None:
        return 1    
    
    analysis = TextBlob(text)
    sentiment = analysis.sentiment.polarity
    
    if sentiment < -0.1:  
        return 0
    elif sentiment > 0.1:  
        return 2
    else:  
        return 1

In [33]:
df_reviews['review'] = df_reviews['review'].astype(str)

In [34]:
df_reviews['sentiment_analysis'] = df_reviews.review.apply(analyze_sentiment)

In [35]:
df_reviews = df_reviews.drop(columns=['review'])
df_reviews.columns 

Index(['user_id', 'user_url', 'funny', 'posted', 'last_edited', 'item_id',
       'helpful', 'recommend', 'sentiment_analysis'],
      dtype='object')

## Dataframe preparation to load them in the API

Now, we are going to preparate and make the Datasets to be in the API. We make join the dataframes that we need and ignore unnecesary columns 

In [188]:
df_games.head()

,publisher,release_date,price,early_access,id,developer,genre,app_title
0,Kotoshiro,2018-01-04,4.99,0.0,761140.0,Kotoshiro,Action,Lost Summoner Kitty
1,Kotoshiro,2018-01-04,4.99,0.0,761140.0,Kotoshiro,Casual,Lost Summoner Kitty
2,Kotoshiro,2018-01-04,4.99,0.0,761140.0,Kotoshiro,Indie,Lost Summoner Kitty
3,Kotoshiro,2018-01-04,4.99,0.0,761140.0,Kotoshiro,Simulation,Lost Summoner Kitty
4,Kotoshiro,2018-01-04,4.99,0.0,761140.0,Kotoshiro,Strategy,Lost Summoner Kitty


In [189]:
price = df_games[["price","id","release_date"]]
price = price.drop_duplicates()
price.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32131 entries, 0 to 85619
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         32131 non-null  object 
 1   id            32131 non-null  float64
 2   release_date  32131 non-null  object 
dtypes: float64(1), object(2)
memory usage: 1004.1+ KB


We see that we have a lot of columns with the name Free or Free To Play, we are going to reeplace them with 0

In [190]:
price ["price"] = price["price"].replace(["Free","Free To Play",'Free to Play'],0)

Now we extract the year and make a new column with it, dropping the original date column

In [191]:
mask = (price['release_date'] != 'No data') #We drop the rows that have no year, since we are not gonna use them
price = price[mask]

price['year'] = price['release_date'].str.extract(r'(\d{4})')
price.drop(columns=['release_date'], inplace=True)
price

,price,id,year
0,4.99,761140.0,2018
5,0,643980.0,2018
9,0,670290.0,2017
14,0.99,767400.0,2017
21,3.99,772540.0,2018
...,...,...,...
85603,1.99,745400.0,2018
85607,1.99,773640.0,2018
85611,4.99,733530.0,2018
85614,1.99,610660.0,2018


In [192]:
df_item_new_df = df_items[["item_id","items_count"]]
df_item_new_df = df_item_new_df.rename(columns={'item_id':'id'}) #rename the column to join them later
df_item_new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5094105 entries, 0 to 5094104
Data columns (total 2 columns):
 #   Column       Dtype
---  ------       -----
 0   id           int64
 1   items_count  int64
dtypes: int64(2)
memory usage: 77.7 MB


In [193]:
df_first_query = df_item_new_df.merge(price, on='id', how='left')   #first dataframe to the first query
df_first_query['price'].fillna('0', inplace=True)  #replace the missing values with 0, since we assume that are free to play
df_first_query["price"] = df_first_query['price'].replace("No data",0)

In [194]:
df_first_query = df_first_query.dropna(subset="year") #we drop the null year items since they are useless for this query

In [195]:
df_first_query.head()

,id,items_count,price,year
0,10,277,9.99,2000
1,20,277,4.99,1999
2,30,277,4.99,2003
3,40,277,4.99,2001
4,50,277,4.99,1999


Now we have to do the Query, we have to group by the itemcount by content

In [196]:
grouped = df_first_query.groupby('year')  #first, we group

grouped

In [197]:
# Then, count the total elements by year
total_count_by_year = grouped['items_count'].sum().reset_index()

total_count_by_year

,year,items_count
0,1983,47345
1,1984,13835
2,1987,308269
3,1988,723415
4,1989,470449
5,1990,1303587
6,1991,1065151
7,1992,1383877
8,1993,3135772
9,1994,3821796


In [199]:
# Calculate the number of 0 (free) elements by year
zero_price_count_by_year = grouped.apply(lambda group: (group['price'] == 0).sum()).reset_index()
zero_price_count_by_year

,year,0
0,1983,0
1,1984,0
2,1987,0
3,1988,0
4,1989,0
5,1990,333
6,1991,330
7,1992,0
8,1993,165
9,1994,165


In [203]:
# We calculate the percentange
percentage_zero_price_by_year = (zero_price_count_by_year[0] / total_count_by_year['items_count']) * 100
percentage_zero_price_by_year

0     0.000000
1     0.000000
2     0.000000
3     0.000000
4     0.000000
5     0.025545
6     0.030982
7     0.000000
8     0.005262
9     0.004317
10    0.003132
11    0.004392
12    0.000000
13    0.008819
14    0.050673
15    0.000000
16    0.001441
17    0.000000
18    0.010092
19    0.000000
20    0.089080
21    0.024296
22    0.012972
23    0.017162
24    0.027480
25    0.013023
26    0.043761
27    0.065647
28    0.047795
29    0.077550
30    0.100738
31    0.072162
32    0.455502
33    0.000000
dtype: float64

In [207]:
#We create the definitive data frame
result_df = pd.DataFrame({
    'Year': total_count_by_year['year'],
    'Total Items': total_count_by_year['items_count'],
    'Zero Price Items': zero_price_count_by_year[0],
    'Percentage of Zero Price': percentage_zero_price_by_year
})
result_df

,Year,Total Items,Zero Price Items,Percentage of Zero Price
0,1983,47345,0,0.000000
1,1984,13835,0,0.000000
2,1987,308269,0,0.000000
3,1988,723415,0,0.000000
4,1989,470449,0,0.000000
5,1990,1303587,333,0.025545
6,1991,1065151,330,0.030982
7,1992,1383877,0,0.000000
8,1993,3135772,165,0.005262
9,1994,3821796,165,0.004317


In [208]:
result_df.to_csv('Api_DataFrame/developer.csv', index=False, encoding='utf-8')
print('developer.csv was saved')

developer.csv was saved
